## 1.1 Packages & Libraries

In [145]:
#!pip install split-folders
#!pip install pydot
#!pip install graphviz
#!pip install keras
#!pip install visualkeras

import numpy as np
import os
import glob
import pandas as pd
import cv2
from tqdm import tqdm
import io
import tensorflow.keras
import sklearn
import pydot
import graphviz
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.optim.lr_scheduler import StepLR

import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import visualkeras

from sklearn.utils import shuffle
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout, Dense, Activation
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix
import IPython
import scikitplot as skplt

## 1.2 Specify a classification class

In [146]:
categories = ['glioma', 'meningioma', 'notumor', 'pituitary']

## 1.3 Data Split (training:test = 8:2)

In [ ]:
#import splitfolders
#splitfolders.ratio("C:/archive/Training", output="C:/archive/Out", seed=1234, ratio=(.8, .2))

## 1.4.1 Building 'ImageDataGenerator(IDG)'

In [41]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255 # rescale
)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
     rescale=1./255) # rescale

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
     rescale=1./255) # rescale

Copying files: 5712 files [00:23, 243.57 files/s]


## 1.4.2 Apply IDG by data set

In [212]:
train_dir = "C:/archive/Train"
validation_dir = "C:/archive/Validation"
test_dir = "C:/archive/Test"

train_generator = train_datagen.flow_from_directory(
        # 타겟 디렉터리
        train_dir,
        target_size =(150, 150),
        batch_size=15,
        class_mode='categorical') #주어진 데이터는 4개의 클래스가 있으므로 'categorical'

validation_generator = validation_datagen.flow_from_directory(
        # 타겟 디렉터리
        validation_dir,
        target_size =(150, 150),
        batch_size=15,
        class_mode='categorical') #주어진 데이터는 4개의 클래스가 있으므로 'categorical'

test_generator = test_datagen.flow_from_directory(
        # 타겟 디렉터리
        test_dir,
        target_size =(150, 150),
        batch_size=15,
        class_mode='categorical') #주어진 데이터는 4개의 클래스가 있으므로 'categorical'

x_train, y_train = train_generator.next()
x_validation, y_validation = validation_generator.next()
x_test, y_test = test_generator.next()

Found 4568 images belonging to 4 classes.
Found 1144 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


## 3.6.1 'Mini-ResNet' Model<Br>
Architecture: Conv-1 + identity Block(BasicBlock + short-cut) + GlobalAveragePooling + F.C layer 

In [243]:
# 1.기본 블럭 설정
class IdentityBlock(tf.keras.Model):
    def __init__(self, filters, kernel_size):
        super(IdentityBlock, self).__init__(name='')
        
        self.conv1 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.bn1 = tf.keras.layers.BatchNormalization()
        
        self.conv2 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
        self.bn2 = tf.keras.layers.BatchNormalization()
        
        self.relu = tf.keras.layers.Activation('relu')
        self.add = tf.keras.layers.Add()
    
    def call(self, inputs): # x 그대로 출력 후, F(x) + x 출력
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.relu(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        
        x = self.add([x, inputs])
        x = self.relu(x)
        
        return x

In [244]:
# 2.ResNet_구축

class ResNet(tf.keras.Model):
    def __init__(self, num_classes):
        super(ResNet, self).__init__()
        self.conv = tf.keras.layers.Conv2D(64, 7, padding='same')
        self.bn = tf.keras.layers.BatchNormalization()
        self.relu = tf.keras.layers.Activation('relu')
        self.max_pool = tf.keras.layers.MaxPool2D((3, 3))
        self.id1a = IdentityBlock(64, 3)
        self.id1b = IdentityBlock(64, 3)
        self.global_pool = tf.keras.layers.GlobalAveragePooling2D()
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')
    
    def call(self, inputs):
        x = self.conv(inputs)
        x = self.bn(x)
        x = self.relu(x)
        x = self.max_pool(x)
        
        x = self.id1a(x)
        x = self.id1b(x)
        
        x = self.global_pool(x)
        return self.classifier(x)

## 3.6.2 'Mini-ResNet' Model evaluation

In [252]:
resnet = ResNet(4)
resnet.compile(optimizer='adam', loss='categorical_crossentropy',
               metrics=['acc'])
resnet.fit(x_train, y_train, epochs=10)
resnet.evaluate(x_test, y_test)

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 1.4641 - acc: 0.3333
Epoch 2/10
1/1 [==============================] - 1s 904ms/step - loss: 0.7921 - acc: 0.7333
Epoch 3/10
1/1 [==============================] - 1s 925ms/step - loss: 0.6203 - acc: 0.7333
Epoch 4/10
1/1 [==============================] - 1s 978ms/step - loss: 0.5369 - acc: 0.8000
Epoch 5/10
1/1 [==============================] - 1s 886ms/step - loss: 0.4703 - acc: 0.9333
Epoch 6/10
1/1 [==============================] - 1s 1s/step - loss: 0.4190 - acc: 0.9333
Epoch 7/10
1/1 [==============================] - 1s 1s/step - loss: 0.3758 - acc: 0.9333
Epoch 8/10
1/1 [==============================] - 1s 921ms/step - loss: 0.3406 - acc: 0.9333
Epoch 9/10
1/1 [==============================] - 1s 1s/step - loss: 0.3111 - acc: 0.9333
Epoch 10/10
1/1 [==============================] - 1s 856ms/step - loss: 1.4901 - acc: 0.2000


[1.4901173114776611, 0.20000000298023224]